# Scraping USPTO tables

The data for this visualization lives in 51 different HTML tables (50 states + Washington DC), each on a separate page. We could copy and paste each table into an Excel or LibreOffice spreadsheet, unpivot the tables and export to a CSV format, but that doesn’t scale well, and worse, it’s boring. Let's write a script!

#### Importing necessary packages
We first import the necessary packages. For this script we use the collections, csv, requests, and Beautiful Soup to:
* Access the url where the different tables are stored,
* Parse and scrape the resulting page,
* Create a named tuple
* Write to a CSV file

In [0]:
from collections import namedtuple
import csv
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

#### Declaring initial variables
We now declare the variables that will be needed when we scrape USPTO.
1. A dictionary with the full state name to state abbreviation key-value pair,
2. the prefix and suffix url to the HTML tables,
3. the empty list to store scraped values,
4. and the named tuple function to pull the values from the scraped HTML.

In [0]:
# US state codes; add territory codes here if desired
REGION_CODES = {
    'Alabama' : 'AL',
    'Alaska' : 'AK',
    'Arizona' : 'AZ',
    'Arkansas' : 'AR',
    'California' : 'CA',
    'Colorado' : 'CO',
    'Connecticut' : 'CT',
    'Delaware' : 'DE',
    'District of Columbia' : 'DC',
    'Florida' : 'FL',
    'Georgia' : 'GA',
    'Hawaii' : 'HI',
    'Idaho' : 'ID',
    'Illinois' : 'IL',
    'Indiana' : 'IN',
    'Iowa' : 'IA',
    'Kansas' : 'KS',
    'Kentucky' : 'KY',
    'Louisiana' : 'LA',
    'Maine' : 'ME',
    'Maryland' : 'MD',
    'Massachusetts' : 'MA',
    'Michigan' : 'MI',
    'Minnesota' : 'MN',
    'Mississippi' : 'MS',
    'Missouri' : 'MO',
    'Montana' : 'MT',
    'Nebraska' : 'NE',
    'Nevada' : 'NV',
    'New Hampshire' : 'NH',
    'New Jersey' : 'NJ',
    'New Mexico' : 'NM',
    'New York' : 'NY',
    'North Carolina' : 'NC',
    'North Dakota' : 'ND',
    'Ohio' : 'OH',
    'Oklahoma' : 'OK',
    'Oregon' : 'OR',
    'Pennsylvania' : 'PA',
    'Rhode Island' : 'RI',
    'South Carolina' : 'SC',
    'South Dakota' : 'SD',
    'Tennessee' : 'TN',
    'Texas' : 'TX',
    'Utah' : 'UT',
    'Vermont' : 'VT',
    'Virginia' : 'VA',
    'Washington' : 'WA',
    'West Virginia' : 'WV',
    'Wisconsin' : 'WI',
    'Wyoming' : 'WY'
}

BASE_URL_PREFIX = 'http://www.uspto.gov/web/offices/ac/ido/oeip/taf/stcteca/'
BASE_URL_SUFFIX = 'stcl_gd.htm'

MASTER_LIST = []
StateRow = namedtuple('StateRow', 'state_name tech_code year value')

### Scraping each state

We now can loop through each state abbreviation and build the actual url for each table. We then pass that url path to requests and parse through Beautiful Soup. From that parsed result, we search for a HTML table tag and pull values using our named tuple function. We store that result in our final MASTER_LIST.

In [0]:
# for each state code, generate the target URL and pull the data
for state in sorted(REGION_CODES):
    print('Processing data for ' + state)
    path = BASE_URL_PREFIX + REGION_CODES[state].lower() + BASE_URL_SUFFIX
    r = requests.get(path)
    soup = BeautifulSoup(r.text, "html.parser")

    # skip first and last rows, which are headers and totals respectively
    for table_row in soup.find_all('tr')[1:-1]:
        tech_code = table_row.find('td', style=' text-align: left; ').string.strip()
        year = 1963
        # skip the last element, which is a total; we can aggregate the data ourselves
        for value in table_row.find_all('td', {'style': None})[:-1]:
            row = StateRow(state, tech_code, year, value.string.strip())
            MASTER_LIST.append(row)
            year = year + 1

Processing data for Alabama
Processing data for Alaska
Processing data for Arizona
Processing data for Arkansas
Processing data for California
Processing data for Colorado
Processing data for Connecticut
Processing data for Delaware
Processing data for District of Columbia
Processing data for Florida
Processing data for Georgia
Processing data for Hawaii
Processing data for Idaho
Processing data for Illinois
Processing data for Indiana
Processing data for Iowa
Processing data for Kansas
Processing data for Kentucky
Processing data for Louisiana
Processing data for Maine
Processing data for Maryland
Processing data for Massachusetts
Processing data for Michigan
Processing data for Minnesota
Processing data for Mississippi
Processing data for Missouri
Processing data for Montana
Processing data for Nebraska
Processing data for Nevada
Processing data for New Hampshire
Processing data for New Jersey
Processing data for New Mexico
Processing data for New York
Processing data for North Carolina
Processing data for North Dakota
Processing data for Ohio
Processing data for Oklahoma
Processing data for Oregon
Processing data for Pennsylvania
Processing data for Rhode Island
Processing data for South Carolina
Processing data for South Dakota
Processing data for Tennessee
Processing data for Texas
Processing data for Utah
Processing data for Vermont
Processing data for Virginia
Processing data for Washington
Processing data for West Virginia
Processing data for Wisconsin
Processing data for Wyoming

In [0]:
master_df = pd.DataFrame(MASTER_LIST)
master_df

,state_name,tech_code,year,value
0,Alabama,424,1963,1
1,Alabama,424,1964,1
2,Alabama,424,1965,0
3,Alabama,424,1966,0
4,Alabama,424,1967,2
...,...,...,...,...
989558,Wyoming,726,2011,0
989559,Wyoming,726,2012,0
989560,Wyoming,726,2013,0
989561,Wyoming,726,2014,0


## Scraping Tech Codes and Names

We now have to scrape the associated classifications with their code values from the USPTO site.

#### Declaring initial variables
We declare the following prior to scraping the site:
* URL to the Tech code table,
* final list to store the data,
* and named tuple function to pull the necessary data.

In [0]:
URL = 'http://www.uspto.gov/web/patents/classification/selectnumwithtitle.htm'
TECH_CODES = []
ClassRow = namedtuple('ClassRow', 'class_code class_name')

#### Scraping Tech Code Tables

We pass the URL we defined to requests and parse through Beautiful Soup. We now loop through HTML table tags and pull values based on our named tuple function.

In [0]:
REQUEST = requests.get(URL)
SOUP = BeautifulSoup(REQUEST.text, "html.parser")

print('Scraping data')
for table_row in SOUP.find_all('tr'):
    class_code_tag = table_row.find('td', width='27')

    # not a class_code + name row. skip
    if class_code_tag is None:
        continue

    class_code = class_code_tag.string
    class_name = table_row.find('td', width='532').string
    TECH_CODES.append(ClassRow(class_code, class_name))

Scraping data

In [0]:
tech_codes_df = pd.DataFrame(TECH_CODES)
tech_codes_df

,class_code,class_name
0,002,Apparel
1,004,"Baths, closets, sinks, and spittoons"
2,005,Beds
3,007,Compound tools
4,008,Bleaching and dyeing; fluid treatment and chem...
...,...,...
467,D30,Animal husbandry
468,D32,"Washing, cleaning, or drying machine"
469,D34,Material or article handling equipment
470,D99,Miscellaneous


#### Join the two dataframes

In [0]:
merged_df = master_df.merge(tech_codes_df, 'left', left_on='tech_code', right_on='class_code')
merged_df

,state_name,tech_code,year,value,class_code,class_name
0,Alabama,424,1963,1,424,"Drug, bio-affecting and body treating composit..."
1,Alabama,424,1964,1,424,"Drug, bio-affecting and body treating composit..."
2,Alabama,424,1965,0,424,"Drug, bio-affecting and body treating composit..."
3,Alabama,424,1966,0,424,"Drug, bio-affecting and body treating composit..."
4,Alabama,424,1967,2,424,"Drug, bio-affecting and body treating composit..."
...,...,...,...,...,...,...
989558,Wyoming,726,2011,0,726,Information security
989559,Wyoming,726,2012,0,726,Information security
989560,Wyoming,726,2013,0,726,Information security
989561,Wyoming,726,2014,0,726,Information security


#### Exporting to CSV
We open the file, state_tech.csv and tech_code.csv, for writing and:
1. output the column names for the CSV as the first row,
2. and write each iteration of our final list as another row.

In [0]:
# write out to csv
# with open('./state_tech.csv', 'w', newline='') as out:
#     print('Writing data to ' + out.name)
#     CSV_FILE = csv.writer(out, delimiter=',')
#     CSV_FILE.writerow(['Region', 'Tech Class Code', 'Year', 'Utility Patent Count'])
#     CSV_FILE.writerows(MASTER_LIST)
    
# with open('./tech_code.csv', 'w', newline='') as out:
#     print('Writing data to ' + out.name)
#     CSV_FILE = csv.writer(out, delimiter=',')
#     CSV_FILE.writerow(['Class Code', 'Class Name'])
#     CSV_FILE.writerows(TECH_CODES)